In [ ]:
import pandas as pd
import numpy as np
import re

# Pre Process

In [ ]:
with open("data/hp_en.txt", "r") as f:
    hp_en = f.read()
    
with open("data/hp_fa.txt", "r") as f:
    hp_fa = f.read()

In [ ]:
hp_en_no_space = re.sub(r'\s+', ' ', hp_en).strip()
hp_fa_no_space = re.sub(r'\s+', ' ', hp_fa).strip()

In [ ]:
from hazm import Normalizer, SentenceTokenizer

sentence_tokenizer = SentenceTokenizer()

sentences_en = sentence_tokenizer.tokenize(hp_en_no_space)
sentences_fa = sentence_tokenizer.tokenize(hp_fa_no_space)


In [ ]:
sentences_fa

In [ ]:

normalizer = Normalizer()


normalized_sentences_en = [normalizer.normalize(sentence) for sentence in sentences_en]
normalized_sentences_fa = [normalizer.normalize(sentence) for sentence in sentences_fa]


In [ ]:
normalized_sentences_en

In [ ]:
words_en = [sentence.split(" ") for sentence in normalized_sentences_en]
words_fa = [sentence.split(" ") for sentence in normalized_sentences_fa]

In [ ]:
words_fa[110]

In [ ]:
sents_no_punctuation_en = []
sents_no_punctuation_fa = []
for sent in words_fa:
    sent_arr = []
    for word in sent:
        sent_arr.append(normalizer.remove_specials_chars(word))
    sents_no_punctuation_fa.append(sent_arr)

for sent in words_en:
    sent_arr = []
    for word in sent:
        sent_arr.append(normalizer.remove_specials_chars(word))
    sents_no_punctuation_en.append(sent_arr)
        

In [ ]:
sents_no_punctuation_en[110]

In [ ]:
# so it seems that the prev approach is not working
sents_without_punctuation_fa = [re.sub(r'[^\w\s]', '', sentence) for sentence in normalized_sentences_fa]
sents_without_punctuation_en = [re.sub(r'[^\w\s]', '', sentence) for sentence in normalized_sentences_en]

In [ ]:
sents_without_punctuation_en

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("[" 
        u"\U0001F600-\U0001F64F" # emoticons
        u"\U0001F300-\U0001F5FF" # symbols & pictographs
        u"\U0001F680-\U0001F6FF" # transport & map symbols
        u"\U0001F1E0-\U0001F1FF" # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

sents_no_emji_fa = [remove_emoji(sentence) for sentence in sents_without_punctuation_fa]
sents_no_emji_en = [remove_emoji(sentence) for sentence in sents_without_punctuation_en]

In [ ]:
sents_no_emji_en

In [ ]:
sents_no_emji_fa

In [ ]:
from hazm import Lemmatizer, word_tokenize

normalized_fa = normalizer.normalize(hp_fa)
tokenized_fa = [word_tokenize(sentence) for sentence in sents_no_emji_fa]

In [ ]:
lemmatizer = Lemmatizer()

lemmas = []
for sentence in tokenized_fa:
    arr = []
    for token in sentence:
        arr.append(lemmatizer.lemmatize(token))
    lemmas.append(arr)

In [ ]:
type(lemmas[0])

In [ ]:
sents_lower_en = [sentence.lower() for sentence in sentences_en]

In [ ]:
sents_lower_en

In [ ]:
tokenized_lower_en = [word_tokenize(sentence) for sentence in sents_lower_en]

In [ ]:
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

def remove_urls(text):
    return re.sub(r'http[s]?://\S+|www\.\S+', '', text)

In [ ]:
no_num_url_en = [remove_numbers(remove_urls(sentence)) for sentence in sents_lower_en]

In [ ]:
no_num_url_en

In [ ]:
no_num_url_punc_en = [re.sub(r'[^\w\s]', '', sentence) for sentence in no_num_url_en]

In [ ]:
no_num_url_punc_en

# WordCloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
no_num_url_punc_text_en = " ".join(no_num_url_punc_en)

In [ ]:
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(no_num_url_punc_text_en)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()


# word to num

In [ ]:
def words_to_numbers(text:str):
    units = {
        'صفر': 0, 'یک': 1, 'دو': 2, 'سه': 3, 'چهار': 4, 'پنج': 5,
        'شش': 6, 'هفت': 7, 'هشت': 8, 'نه': 9, 'ده': 10, 'یازده': 11,
        'دوازده': 12, 'سیزده': 13, 'چهارده': 14, 'پانزده': 15, 'شانزده': 16,
        'هفده': 17, 'هجده': 18, 'نوزده': 19
    }

    tens = {
        'بیست': 20, 'سی': 30, 'چهل': 40, 'پنجاه': 50,
        'شصت': 60, 'هفتاد': 70, 'هشتاد': 80, 'نود': 90
    }

    hundreds = {
        'صد': 100, 'دویست': 200, 'سیصد': 300, 'چهارصد': 400,
        'پانصد': 500, 'ششصد': 600, 'هفتصد': 700, 'هشتصد': 800, 'نهصد': 900
    }

    scales = {
        'هزار': 1000, 'میلیون': 1000000, 'میلیارد': 1000000000
    }

    words = text.split()
    current_number = 0
    total = 0

    for word in words:
        if word in units:
            current_number += units[word]
        elif word in tens:
            current_number += tens[word]
        elif word in hundreds:
            current_number += hundreds[word]
        elif word in scales:
            current_number = current_number if current_number else 1
            total += current_number * scales[word]
            current_number = 0
        elif word == 'و':
            continue
        else:
            raise ValueError

    total += current_number
    return total

In [ ]:
text = "پنجاه و دو هزار و سیصد و بیست و یک" 
number = words_to_numbers(text)
print(number)

# Spell Check and Correct

In [ ]:
vocabs = pd.read_csv("data/Vocabulary.txt", header=None, names=["word"])

In [ ]:
vocabs

In [ ]:
from hazm import Normalizer, word_tokenize
from spellchecker import SpellChecker

spell = SpellChecker()

In [ ]:
text = """According to a researchwe at an english university, it doesn't mattere in what order the letters in a word are, the only important thing is that fst and last letter is at the right place. The rewt can bee a total mess and you can stil read it without problem. This is because we do not read evry letter by ielf but the word as a whole. Cheerio.
"""

tokenized = word_tokenize(text)
normalizer = Normalizer()
normalized = [normalizer.normalize(word) for word in tokenized]
no_puncs = [re.sub(r'[^\w\s]', '', sentence) for sentence in normalized]

In [ ]:
misspelled = [word for word in no_puncs if word not in vocabs["word"]]

In [ ]:
for word in misspelled:
    print(word)

In [ ]:
corrections = {word: spell.correction(word) for word in misspelled}

In [ ]:
corrections

In [ ]:
corrected_tokenized = tokenized
for word, correction in corrections.items():
    if word in tokenized:
        corrected_tokenized[tokenized.index(word)] = correction

In [ ]:
corrected_text = " ".join(corrected_tokenized)

In [ ]:
corrected_text

## bulding the func

In [ ]:
def correct_spelling(text):
    tokenized = word_tokenize(text)
    normalizer = Normalizer()
    normalized = [normalizer.normalize(word) for word in tokenized]
    no_puncs = [re.sub(r'[^\w\s]', '', sentence) for sentence in normalized]

    misspelled = [word for word in no_puncs if word not in vocabs["word"]]

    corrections = {word: spell.correction(word) for word in misspelled}

    corrected_tokenized = tokenized
    for word, correction in corrections.items():
        if word in tokenized:
            corrected_tokenized[tokenized.index(word)] = correction

    corrected_text = " ".join(corrected_tokenized)
    
    return corrected_text

In [ ]:
corrected_text = correct_spelling(text)
print(corrected_text)